<a href="https://colab.research.google.com/github/evakaldy/Bootcamp_Final_Project/blob/eva_database_exploration/explore_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
# Replace 'data.db' with the path to your database file
db_path = "data.db"  # Local file path
connection = sqlite3.connect(db_path)